In [ ]:
from shutil import copyfile
import cv2
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from PIL import Image

In [ ]:
## train path => /kaggle/input/cassava-leaf-disease-classification/train_images
## train csv path => /kaggle/input/cassava-leaf-disease-classification/train.csv
## test images => /kaggle/input/cassava-leaf-disease-classification/test_images
trainFrame = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')
trainFrame.label = trainFrame.label.astype(str)

mp1 = {'0': 'Cassava Bacterial Blight (CBB)',
 '1': 'Cassava Brown Streak Disease (CBSD)',
 '2': 'Cassava Green Mottle (CGM)',
 '3': 'Cassava Mosaic Disease (CMD)',
 '4': 'Healthy'}

trainFrame.label = trainFrame.label.map(mp1)

#uniqueBreeds = set (trainFrame['label'].values)
#print ('no. of breeds:{}'.format(len(uniqueBreeds)))

#os.makedirs('../trainDir')
#for mem in uniqueBreeds:
#    os.makedirs('../trainDir/' + str(mem))

    
#balanceCounter = 0

#for i in range(len(trainFrame)):
#    imgName = trainFrame.loc[i].values[0]
#    dirName = trainFrame.loc[i].values[1]
    
    #if dirName == 3:
        #if balanceCounter < 5000:
            #balanceCounter += 1
        #else:
            #continue
    
    
#    sourcePathImg = '/kaggle/input/cassava-leaf-disease-classification/train_images/'
#    destPathImg = '../trainDir/'
    
#    sourcePathImg = sourcePathImg + imgName 
#    destPathImg = destPathImg + str(dirName) + '/' + imgName 
#    print (sourcePathImg + '   === >' + destPathImg)
    
#    copyfile(sourcePathImg, destPathImg)

data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.,
    validation_split=0.30,
    rotation_range=360,
    horizontal_flip=True,
    vertical_flip=True,
)

train_data_loader = data_generator.flow_from_dataframe(
    trainFrame,
    directory= '/kaggle/input/cassava-leaf-disease-classification/train_images',
    x_col="image_id",
    y_col="label",
    target_size=(224, 224),
    subset='training',
    class_mode = "sparse"
)

val_data_loader = data_generator.flow_from_dataframe(
    trainFrame,
    directory= '/kaggle/input/cassava-leaf-disease-classification/train_images',
    x_col="image_id",
    y_col="label",
    target_size=(224, 224),
    subset='validation',
    class_mode = "sparse"
)



In [ ]:
#import matplotlib.pyplot as plt
#import tensorflow as tf
from tensorflow.keras import layers

#batch_size = 100
#img_height = 256
#img_width = 256

#training_ds = tf.keras.preprocessing.image_dataset_from_directory(
 #   '../trainDir',
  #  validation_split=0.2,
   # subset= "training",
    #seed=42,
    #image_size= (img_height, img_width),
    #batch_size=batch_size

#)

#validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
#   '../trainDir',
#    validation_split=0.2,
#    subset= "validation",
#    seed=42,
#    image_size= (img_height, img_width),
#    batch_size=batch_size

#)

In [ ]:
#class_names = training_ds.class_names
#AUTOTUNE = tf.data.experimental.AUTOTUNE
#training_ds = training_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
## Defining Cnn
## lets define our CNN
### this model will not be used but other will
MyCnn1 =  tf.keras.models.Sequential([
    layers.Input(shape = (224, 224, 3), name = 'I1'),
    layers.BatchNormalization(),
    layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    
    layers.Dense(5, activation='softmax')
])
MyCnn1.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

#preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
preprocess_input = tf.keras.applications.efficientnet.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

IMG_SHAPE = (224, 224) + (3,)
base_model = tf.keras.applications.EfficientNetB7(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='../input/tfkerasefficientnetimagenetnotop/efficientnetb7_notop.h5')

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(5)
base_model.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = rescale(x)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(.02)(x)
outputs = prediction_layer(x)
MyCnn = tf.keras.Model(inputs, outputs)

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(MyCnn, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * 0.3

    
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
## training our model 
MyCnn.compile(optimizer= tf.keras.optimizers.Adam(lr=2e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
retVal = MyCnn.fit(train_data_loader, validation_data= val_data_loader,callbacks=[callback], epochs = 4)

In [ ]:
plt.plot(retVal.history['loss'], label = 'training loss')
plt.plot(retVal.history['accuracy'], label = 'training accuracy')
plt.legend()

In [ ]:
plt.plot(retVal.history['val_loss'], label = 'validation loss')
plt.plot(retVal.history['val_accuracy'], label = 'validation accuracy')
plt.legend()

In [ ]:
AnsFrame = {'image_id' : [],
            'label' : []
           }

def predictor():
    path = '/kaggle/input/cassava-leaf-disease-classification/test_images'
    files = os.listdir(path)
    for mem in files:
        img = Image.open(path + '/' + mem)
        img = cv2.resize(np.asarray(img),(224, 224))
        img = np.array([img/255])
        snakeName = np.argmax (MyCnn.predict(img))
        AnsFrame['image_id'].append(mem)
        AnsFrame['label'].append (snakeName)
        #print (snakeName)

In [ ]:
predictor()

In [ ]:
AnsFrame = pd.DataFrame(AnsFrame)
AnsFrame.to_csv('submission.csv', index= False)

In [ ]:
AnsFrame